In [1]:
import os
import openai
os.environ['CUDA_VISIBLE_DEVICES'] = '00'
os.environ['CUDA_LAUNCH_BLOCKING'] = '00'
os.environ['HUGGINGFACEHUB_API_TOKEN'] = '00'
os.environ["OPENAI_API_KEY"] = "00"  
openai.api_key = "00"

In [2]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
import pinecone 
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from llama_index import LLMPredictor

/workspace/JupyterLab (DATA)/NLP_GPT/nlp_gpt/lib/python3.8/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


# Import Data

In [3]:
directory = 'data/'

def load_docs(directory):
    loader = DirectoryLoader(directory)
    documents = loader.load()
    return documents

documents = load_docs(directory)
len(documents)

1

# Text Splitter

In [4]:
def split_docs(documents,chunk_size=1000,chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(documents)
    return docs

docs = split_docs(documents)
print(len(docs))

22


# Words Embedding

In [5]:
embeddings = HuggingFaceEmbeddings()

query_result = embeddings.embed_query("Hello world")
len(query_result)

768

# Using Pincone

In [21]:
pinecone.init(
    api_key="5deaf73e-7868-4138-a662-72738d095818",  # find at app.pinecone.io
    environment="northamerica-northeast1-gcp"  # next to api key in console
)

index_name = "langchain-demo"

index = Pinecone.from_documents(docs, embeddings, index_name=index_name)

# Text Similarity

In [19]:
def get_similiar_docs(query,k=3,score=False):
    if score:
        similar_docs = index.similarity_search_with_score(query,k=k)
    else:
        similar_docs = index.similarity_search(query,k=k)
    return similar_docs

query = "What is specification of Iphone 14?"
similar_docs = get_similiar_docs(query, score=True)
similar_docs

[(Document(page_content='77.6 mm Height: (3.05 160.7 mm (6.33 inches) Depth: 7.85 mm\n\nHeight: (2.81 147.5 mm (5.81 inches) Depth: 7.85 mm\n\n\n\n\n\nThe iPhone 14 Pro display has rounded corners that follow a beautiful curved design, and these corners are within a standard rectangle. When measured as a standard rectangular shape, the screen is 15.54 centimetres / 6.12 inches diagonally (actual viewable area is less).\n\nThe iPhone 14 Pro Max display has rounded corners that follow a beautiful curved design, and these corners are within a standard rectangle. When measured as a standard rectangular shape, the screen is 17.00 centimetres / 6.69 inches diagonally (actual viewable area is less).\n\nSupport for display of multiple languages and characters simultaneously\n\nRated IP68 (maximum depth of 6 metres up to 30 minutes) under IEC standard 60529\n\nPro camera system\n\nMain: 24 mm, ƒ/1.78 aperture, second-generation sensor-shift optical image stabilisation, lens, 100% Focus Pixels',

# Build LLM

In [8]:
model_name = "text-davinci-003"
llm = OpenAI(model_name=model_name)

In [9]:
template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])

In [10]:
chain = load_qa_chain(llm, chain_type="stuff")

def get_answer(query):
    similar_docs = get_similiar_docs(query)
    answer =  chain.run(input_documents=similar_docs, question=query)
    return  answer

In [11]:
query = "is iPhone 14 Pro and iPhone 14 Pro Max are splash, water and dust resistant?"  
get_answer(query)

' Yes, iPhone 14 Pro and iPhone 14 Pro Max are splash, water and dust resistant, and were tested under controlled laboratory conditions with a rating of IP68 under IEC 60529 (maximum depth of 6 metres up to 30 minutes).'

In [12]:
print(get_answer(query))

 Yes, iPhone 14 Pro and iPhone 14 Pro Max are splash, water and dust resistant and were tested under controlled laboratory conditions with a rating of IP68 under IEC 60529 (maximum depth of 6 metres up to 30 minutes). However, resistance might decrease as a result of normal wear and liquid damage is not covered under warranty.
